In [1]:
#Import libraries
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings("ignore")




In [2]:
df=pd.read_csv('data.csv',encoding='ISO-8859-1',dtype={'CustomerID': str,'InvoiceID': str})
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom


# **Data Preparation and Preprocessing**

In [3]:
def check_df(dataframe):
    print("################ Shape ####################")
    print(dataframe.shape)
    print("############### Columns ###################")
    print(dataframe.columns)
    print("############### Types #####################")
    print(dataframe.dtypes)
    print("############### Head ######################")
    print(dataframe.head())
    print("############### Tail ######################")
    print(dataframe.tail())
    print("############### Describe ###################")
    print(dataframe.describe().T)

check_df(df)

################ Shape ####################
(541909, 8)
############### Columns ###################
Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')
############### Types #####################
InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID      object
Country         object
dtype: object
############### Head ######################
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

      InvoiceDate  UnitPrice CustomerI

# **Answering Some Questions About Data**

In [4]:
#Are there any missing observations in the dataset?
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [5]:
#Remove the missing observations from the dataset
df.dropna(inplace=True)

In [6]:
#How many unique items in the dataset?
df["Description"].nunique()

3896

In [7]:
#How many product in the dataset?
df["Description"].value_counts()

WHITE HANGING HEART T-LIGHT HOLDER    2070
REGENCY CAKESTAND 3 TIER              1905
JUMBO BAG RED RETROSPOT               1662
ASSORTED COLOUR BIRD ORNAMENT         1418
PARTY BUNTING                         1416
                                      ... 
ANTIQUE RASPBERRY FLOWER EARRINGS        1
WALL ART,ONLY ONE PERSON                 1
GOLD/AMBER DROP EARRINGS W LEAF          1
INCENSE BAZAAR PEACH                     1
PINK BAROQUE FLOCK CANDLE HOLDER         1
Name: Description, Length: 3896, dtype: int64

In [8]:
#Rank the 5 most ordered products from most to least
df.groupby("Description").agg({"Quantity":"sum"}).sort_values("Quantity", ascending=False).head(5)

,Quantity
Description,
WORLD WAR 2 GLIDERS ASSTD DESIGNS,53215
JUMBO BAG RED RETROSPOT,45066
ASSORTED COLOUR BIRD ORNAMENT,35314
WHITE HANGING HEART T-LIGHT HOLDER,34147
PACK OF 72 RETROSPOT CAKE CASES,33409


In [11]:
#Remove the canceled transactions from the dataset
df = df[~df["InvoiceNo"].str.contains("C", na=False)]

In [12]:
#The total amount of each invoice and we create that column just multiply 'Price' and 'Quantity'
df["TotalPrice"] = df["Quantity"] * df["UnitPrice"]

# **Calculating of RFM Metrics**

In [16]:
#Determining the analysis date for the recency 
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df["InvoiceDate"].max()
today_date = dt.datetime(2011, 12, 11)
rfm = df.groupby('CustomerID').agg({'InvoiceDate': lambda date: (today_date - date.max()).days,
                                     'InvoiceNo': lambda num: num.nunique(),
                                     'TotalPrice': lambda TotalPrice: TotalPrice.sum()})
rfm.columns=["Recency","Frequency","Monetary"]
rfm = rfm[rfm["Monetary"] > 0]
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
Recency,4338.0,93.059474,100.012264,1.00,18.000,51.000,142.75,374.00
Frequency,4338.0,4.272706,7.706221,1.00,1.000,2.000,5.00,210.00
Monetary,4338.0,2054.266460,8989.230441,3.75,307.415,674.485,1661.74,280206.02


# **Converting RFM Scores to Single Variable**


In [17]:
#Date from customer's last purchase.The nearest date gets 5 and the furthest date gets 1.
rfm["recency_score"] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])
# Total number of purchases.The least frequency gets 1 and the maximum frequency gets 5.
rfm["frequency_score"] = pd.qcut(rfm["Frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
#Total spend by the customer.The least money gets 1, the most money gets 5.
rfm["monetary_score"]= pd.qcut(rfm["Monetary"],5,labels=[1,2,3,4,5])
rfm.head()

,Recency,Frequency,Monetary,recency_score,frequency_score,monetary_score
CustomerID,,,,,,
12346,326,1,77183.60,1,1,5
12347,3,7,4310.00,5,5,5
12348,76,4,1797.24,2,4,4
12349,19,1,1757.55,4,1,4
12350,311,1,334.40,1,1,2


In [18]:
#RFM - The value of 2 different variables that were formed was recorded as a RFM_SCORE
rfm["RFM_SCORE"] = (rfm["recency_score"].astype(str) + rfm["frequency_score"].astype(str))

# **Segmenting Customers Using RFM Score**

In [19]:
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}
rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True)
rfm.head()

,Recency,Frequency,Monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
CustomerID,,,,,,,,
12346,326,1,77183.60,1,1,5,11,hibernating
12347,3,7,4310.00,5,5,5,55,champions
12348,76,4,1797.24,2,4,4,24,at_Risk
12349,19,1,1757.55,4,1,4,41,promising
12350,311,1,334.40,1,1,2,11,hibernating


# **Interpretation of Descriptive Statistics of Segments**

In [20]:
rfm[["segment", "Recency","Frequency","Monetary"]].groupby("segment").agg(["mean","count","max"]).round()

Recency            Frequency            Monetary        \
                       mean count  max      mean count  max     mean count   
segment                                                                      
about_to_sleep         53.0   352   72       1.0   352    2    472.0   352   
at_Risk               154.0   593  374       3.0   593    6   1085.0   593   
cant_loose            133.0    63  373       8.0    63   34   2796.0    63   
champions               6.0   633   13      12.0   633  210   6858.0   633   
hibernating           218.0  1071  374       1.0  1071    2    489.0  1071   
loyal_customers        34.0   819   72       6.0   819   63   2864.0   819   
need_attention         52.0   187   72       2.0   187    3    898.0   187   
new_customers           7.0    42   13       1.0    42    1    388.0    42   
potential_loyalists    17.0   484   33       2.0   484    3   1041.0   484   
promising              24.0    94   33       1.0    94    1    294.0    94   

                               
                          max  
segment                        
about_to_sleep         6208.0  
at_Risk               44534.0  
cant_loose            10254.0  
champions            280206.0  
hibernating           77184.0  
loyal_customers      124915.0  
need_attention        12602.0  
new_customers          3861.0  
potential_loyalists  168472.0  
promising              1758.0

# **Reviews About the Segments**

**Cant Loose**

* There are 63 people in this segment.
* Shopping was done on average 133 days ago.
* The frequency of shopping is 8, the total number of purchases is 63.
* A total of £102,54 has been spent.

**Action:** Even if the last purchase was made 133 days, the total number of purchases is high. It is a group of customers who do not come for a long time, but also make a lot of purchases when they come. We can analyze the process by sending surveys to these customers, and we can be changed by sharing personalized campaigns by e-mail.

**Need Attention**

* There are 187 people in this segment.
* Shopping was done on average 52 days ago.
* The frequency of shopping is 2, the total number of purchases is 3.
* A total of £12,602 has been spent.

**Action:** These customers need to be reminded of the brand.So, short-term discounts can be made to remind these customers of our brand and allow them to shop again.
